In [1]:
import cv2
import os
import torch
import math
import numpy as np
import torchvision
import glob
bpath = os.path.abspath(os.getcwd())

In [2]:
print(bpath)

/Users/leiwang/Downloads/taylor-videos-main/taylor


In [3]:
def taylorvideo(video_path, terms, window_size, step_size):
    
    if window_size - 3 < terms:
        print("The given temporal block length is not enough to compute K terms.")
        
    else:
        vidcap = cv2.VideoCapture(video_path)
        
        fps = vidcap.get(cv2.CAP_PROP_FPS)
        vlen = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
        print("Video length: %d" % vlen, " |  FPS: %d" % fps)
        
        success, image = vidcap.read()
        # print(image.shape, ' === initial frame')
        count = 1
        while success:
            
            if count < window_size:
                success,image = vidcap.read()
                # print('Read a new frame: ', success, ' This is just a count: ', count)
                count += 1

In [4]:
def videoConvert(vid_path,o,terms,tPrime):
    if (tPrime <= terms+3):
        tPrime = terms + 3
        print("Invalid sub-sequence length, auto set to:")
        print(tPrime)
    cap = cv2.VideoCapture(vid_path)
    fpsT = cap.get(cv2.CAP_PROP_FPS)
    vidlength = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print("length = ")
    print(vidlength)
    print("fps = ")
    print(fpsT)
    ret,frame = cap.read()
    norm_g = torch.from_numpy(cv2.cvtColor(frame,  cv2.COLOR_BGR2GRAY))
    norm_g = torch.div(norm_g, 255.0)
    h, w = norm_g.shape
    
    length = terms + 3
    full_difference_list = torch.zeros((length,length,h,w), dtype=torch.float64)
    full_difference_list[0,0,:,:] = norm_g

    for initialInc in range(1,terms+3):
        ret, frame = cap.read()
        norm_g = torch.from_numpy(cv2.cvtColor(frame,  cv2.COLOR_BGR2GRAY))
        norm_g = torch.div(norm_g, 255.0)
        full_difference_list[0,initialInc,:,:] = norm_g

    img = torch.zeros(((vidlength-tPrime+1),h,w,3), dtype=torch.float64)
    
    for sequences in range(0,vidlength-tPrime+1): #vidlength-tPrime+1
        print(sequences)
        if sequences == 0:
            for listInc in range(1,terms+3):
                full_difference_list[listInc,:,:,:] = torch.nn.functional.pad((full_difference_list[listInc-1,1:,:,:]-full_difference_list[listInc-1,:-1,:,:]),(0,0,0,0,0,1))
        if sequences != 0:
            ret, frame = cap.read()

            full_difference_list = torch.roll(full_difference_list, -1, 1)
            
            norm_g = torch.from_numpy(cv2.cvtColor(frame,  cv2.COLOR_BGR2GRAY))
            norm_g = torch.div(norm_g, 255.0)
            full_difference_list[0,length-1,:,:] = norm_g
            for listInc in range(1,terms+3):
                full_difference_list[listInc,length-1-listInc,:,:] = full_difference_list[listInc-1,length-listInc,:,:] - full_difference_list[listInc-1,length-1-listInc,:,:]
        
        t1Sum = 0
        t2Sum = 0
        t3Sum = 0

        dummy = full_difference_list[0,0,:,:].unsqueeze(0).repeat(length, 1, 1)
        xa_Tensor =  full_difference_list[0,:,:,:] - dummy

        for incB in range(0,terms):
            part = torch.div(torch.pow(xa_Tensor,incB), math.factorial(incB))
            t1Sum += torch.mul(torch.sum(part,0),full_difference_list[incB+1,0,:,:])
            t2Sum += torch.mul(torch.sum(part,0),full_difference_list[incB+2,0,:,:])
            t3Sum += torch.mul(torch.sum(part,0),full_difference_list[incB+3,0,:,:])

        # R channel
        img[sequences,:,:,0] = t1Sum 
        # G channel
        img[sequences,:,:,1] = t2Sum 
        # B channel
        img[sequences,:,:,2] = t3Sum
    cap.release()

    torchvision.io.write_video(filename=o, video_array=img, fps=fpsT, video_codec='h264')
    print(vid_path)
    print(o)

for file in glob.glob(bpath + '/2/*.mp4',recursive=True):
    print(file, ' ---')
    rel_path = os.path.relpath(file, bpath)

    # Create the corresponding directory structure in Tvideos
    newdir = bpath + '/1terms2' + os.path.dirname(rel_path)
    os.makedirs(newdir, exist_ok=True)

    # Construct the new file path in Tvideos
    file_name = os.path.splitext(os.path.basename(file))[0]
    newpath = os.path.join(newdir, file_name + '.avi')
    print(newpath)

    # Perform the video conversion
    videoConvert(file, newpath, 1)

videoConvert("brush.mp4","brush-new.mp4",3,0)

Invalid sub-sequence length, auto set to:
6
length = 
116
fps = 
24.0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
brush.mp4
brush-new.mp4
